In [1]:
 #get kerberos token for eos
import getpass, os
os.system("echo %s | kinit gandreas@CERN.CH" %getpass.getpass()) 


········


0

In [2]:
input_file_names = ['/eos/lhcb/user/g/gandreas/Lb2Lemu/tuples/S28/MC/Bd2JpsiKS_MC12_Pythia8_MagDown.root']
output_file_names_root = '/eos/lhcb/user/g/gandreas/Lb2Lemu/tuples/S28/MC/Bd2JpsiKS_MC12_Pythia8_MagDown_BDT'
tuple_name = 'Lb2Lmumu_Tuple/DecayTree'

In [3]:
import sys
tag = 0
decay = "emu"
k=5
best_classifier_name="xgboost"
classifiers_dir = 'classifiers/'.format(decay, tag)
classifiers_subdir = "/trained_"+best_classifier_name+str(tag)+"_"+decay+"/"
dir_name = classifiers_dir+classifiers_subdir
if not os.path.exists(dir_name):
    raise Exception("Cannot find the classifiers folder!")
    
best_classifier_name = 'xgboost'

In [4]:
#X-checking
if (len(os.listdir(dir_name))!=k):
    raise Exception("Number of folds != number of classifiers found in", dir_name)

In [5]:
import cuts
cuts = cuts.Lb_Lemu_preselection.GetTitle() #if data!! None, if MC

Welcome to JupyROOT 6.08/02


/cvmfs/lhcb.cern.ch/lib/lcg/releases/LCG_87/ROOT/6.08.02/x86_64-slc6-gcc49-opt/lib/ROOT.py:417: UserWarning: "from ROOT import *" is not supported under IPython
  warnings.warn( '"from ROOT import *" is not supported under IPython' )


In [6]:
#retrieve trained classifiers
import pickle

k_classifiers=[]
for i in range(k):
    file_name = dir_name+best_classifier_name+"_k"+str(i)+".pickle.dat"
    k_classifiers.append(pickle.load(open(file_name, "r")))
    
if os.path.exists(file_name):
        best_classifier = pickle.load(open(file_name, "r"))
else: raise Exception("Cannot find the classifier!")
    
#branches = best_classifier.features ##for now they are set manyally. I'll fix this later byt taking them from vars_db
branches = k_classifiers[0].features
branches = [var.replace("_mathPLUS_", "+").replace("_mathMINUS_", "-").replace("_mathTIMES_", "*").replace("_mathOVER_", "/") for var in branches]
additional_branches = ['Lb_M', 'eventNumber'] #spectator variables and others

In [10]:
from tools.MVA2root import MVA2root

MVA2root (k_classifiers,
          classifier_name=best_classifier_name+str(tag),
          inputfilenames = input_file_names,
          tuplename = tuple_name,
          max_files_atatime = 50,
          branches = branches+additional_branches,
          outfilenameroot = output_file_names_root,
          fix_features_names_for_MVA = True,
          cuts=cuts
         )

ROOTError: level=3000, loc='TTreeFormula::Compile', msg=' Bad numerical expression : "Lb_hop_LoKi_mass_bv"'

ERROR:ROOT.TTreeFormula.Compile: Bad numerical expression : "Lb_hop_LoKi_mass_bv"


Processing chunk 0 of 1 ...
Applying MVA to fold 0 ...
Applying MVA to fold 1 ...
Applying MVA to fold 2 ...
Applying MVA to fold 3 ...
Applying MVA to fold 4 ...
Done!
Writing output file with the following content: 
TFile**		/eos/lhcb/user/g/gandreas/Lb2Lemu/tuples/S28/LbLemu_MC_test_BDT_0.root	
 TFile*		/eos/lhcb/user/g/gandreas/Lb2Lemu/tuples/S28/LbLemu_MC_test_BDT_0.root	
  KEY: TTree	DecayTree;1	DecayTree
